In [1]:
import numpy as np

In [9]:
"""
parallel_scheduler_vFull_idle_fixed.py
────────────────────────────────────────
Same as parallel_scheduler_vFull_idle.py, but now **executes every MS gate** in a layer
(instead of only the first). No other logic is changed.
"""

import numpy as np, networkx as nx
from trap import create_trap_graph

# ── 1) load logical layers -------------------------------------------------
with open("time_steps_raw_fixedphi.txt") as f:
    gate_layers = [eval(l.strip()) for l in f]

L, Q = len(gate_layers), 8

# ── 2) trap & node sets ----------------------------------------------------
G = create_trap_graph()
INT_SITES = {n for n, d in G.nodes(data=True) if d["type"] == "interaction"}
STD_SITES = {n for n, d in G.nodes(data=True) if d["type"] == "standard"}

# ── 3) initial positions ---------------------------------------------------
pos_hist = [[(0, 1), (0, 3), (1, 0), (1, 4),
             (3, 0), (3, 4), (4, 1), (4, 3)]]  # list-of-lists
HOME = {q: pos_hist[0][q] for q in range(Q)}
IDLE = set()                                # qubits parked idle
FORBID = set(HOME.values())

scheduled_ops: list[list[tuple]] = []

# ── helper: tick & clone last row -----------------------------------------
def clone_last_row():
    pos_hist.append(list(pos_hist[-1]))

def tick(ops=None):
    if ops is None:
        ops = []
    scheduled_ops.append(ops)
    clone_last_row()

# ── routing primitives ------------------------------------------------------
def coord(q):
    return pos_hist[-1][q][:2]

def at_idle(q):
    return len(pos_hist[-1][q]) == 3

def wake(q):
    """If q is idle, move it 'down' to HOME[q]' in one tick."""
    if at_idle(q):
        tick()
        row, col, _ = pos_hist[-2][q]
        pos_hist[-1][q] = (row, col)
        IDLE.discard(q)

def hop(cur, goal):
    if cur == goal:
        return None
    def w(u, v, d):
        return 1e6 if v in FORBID and v != goal else 1
    return nx.dijkstra_path(G, cur, goal, weight=w)[1]

def route(goals):
    """One-hop parallel routing, waking any idler involved first."""
    for q in goals:
        wake(q)
    while any(coord(q) != goals[q] for q in goals):
        prop = {q: hop(coord(q), goals[q]) for q in goals}
        occ  = {coord(r): r for r in range(Q) if not at_idle(r)}
        moves, dest = {}, {}
        for q, nxt in prop.items():
            if nxt:
                dest.setdefault(nxt, []).append(q)
        for dst, qs in dest.items():
            qs.sort()
            if dst in occ and occ[dst] not in prop:
                continue
            moves[qs[0]] = dst
        # prevent direct swaps
        killers = {max(q, p)
                   for q, d in moves.items()
                   for p, dp in moves.items()
                   if p < q and d == coord(p) and dp == coord(q)}
        for k in killers:
            moves.pop(k)
        if not moves:
            fb = next((q for q, step in prop.items() if step), None)
            if fb is None:
                break
            moves[fb] = prop[fb]
        tick()
        for q, dst in moves.items():
            pos_hist[-1][q] = dst

# ── look-ahead helper -------------------------------------------------------
def needed_soon(q, depth, lookahead=6):
    for i in range(depth+1, min(L, depth+1+lookahead)):
        for op in gate_layers[i]:
            if op[0] == "MS" and q in op[2]:
                return True
            if op[0] in ("RX", "RY") and q == op[2]:
                return True
    return False

# ── Initial bulk-idle (t=0) ------------------------------------------------
for q in range(Q):
    if not needed_soon(q, -1, 6):
        r, c = pos_hist[0][q]
        pos_hist[0][q] = (r, c, "idle")
        IDLE.add(q)

# ── 4) main scheduler loop -------------------------------------------------
for depth in range(L):
    layer = gate_layers[depth]

    # peek next-MS qubits for reuse
    nxt = next((g for g in gate_layers[depth+1] if g[0]=="MS"),
               None) if depth+1 < L else None
    nxt_qubits = set(nxt[2]) if nxt else set()

    # ── execute every MS in this layer ────────────────────────────────────
    ms_gates = [g for g in layer if g[0]=="MS"]
    for _, ang, (qA, qB) in ms_gates:
        site = min(INT_SITES,
                   key=lambda s: sum(abs(s[0]-coord(q)[0])+
                                     abs(s[1]-coord(q)[1])
                                     for q in (qA, qB)))
        route({qA: site, qB: site})
        scheduled_ops[-1].append(("MS", ang, (qA, qB)))
        tick()   # hold
        shared = {qA, qB} & nxt_qubits
        if not shared:
            route({qA: HOME[qA], qB: HOME[qB]})
        elif len(shared)==1:
            stay = shared.pop()
            leave = qB if stay==qA else qA
            new_q = (nxt_qubits - {stay}).pop()
            route({leave: HOME[leave], new_q: site})
        else:
            nbs = [n for n in G.neighbors(site) if n in STD_SITES][:2]
            route({qA: nbs[0], qB: nbs[1]})

    # ── single-qubit rotations (ASAP) ─────────────────────────────────────
    sq_ops = [(n,a,q) for n,a,q in layer if n in ("RX","RY")]
    if sq_ops:
        exits, occ = {}, set(coord(r) for r in range(Q) if not at_idle(r))
        for _,_,q in sq_ops:
            here=coord(q)
            if here in INT_SITES:
                for nb in G.neighbors(here):
                    if nb in STD_SITES and nb not in occ:
                        exits[q]=nb; occ.add(nb); break
        if exits:
            route(exits)
        if scheduled_ops and not scheduled_ops[-1]:
            scheduled_ops[-1].extend(sq_ops)
        else:
            tick(sq_ops)
        back={q:HOME[q] for _,_,q in sq_ops if q not in nxt_qubits}
        if back:
            route(back)

    # ── idle-store after each layer ------------------------------------------
    for q in range(Q):
        if coord(q)==HOME[q] and not at_idle(q):
            if not needed_soon(q, depth, 6):
                tick()
                r,c=coord(q)
                pos_hist[-1][q]=(r,c,"idle")
                IDLE.add(q)

# ── final single-tick wake of all idlers ----------------------------------
if IDLE:
    tick()
    for q in list(IDLE):
        r,c,_=pos_hist[-1][q]
        pos_hist[-1][q]=(r,c)
    IDLE.clear()

# ── expose outputs ---------------------------------------------------------
pos_sequence = np.array(pos_hist, dtype=object)
print("Total ticks:", len(scheduled_ops))
print("pos_sequence shape:", pos_sequence.shape)


Total ticks: 448
pos_sequence shape: (449, 8)


In [10]:
from verifier import verifier
from fidelity import fidelity
import trap

pos_sequence = [
    [tuple(pos) for pos in pos_hist[t+1]]
    for t in range(len(scheduled_ops))
]

print(pos_sequence)
print(scheduled_ops)

graph = trap.create_trap_graph()

verifier(pos_sequence, scheduled_ops, graph)
fidelity(pos_sequence, scheduled_ops, graph)

[[(0, 1, 'idle'), (0, 3, 'idle'), (1, 0, 'idle'), (1, 4, 'idle'), (3, 0, 'idle'), (3, 4, 'idle'), (4, 1), (4, 3)], [(0, 1, 'idle'), (0, 3, 'idle'), (1, 0, 'idle'), (1, 4, 'idle'), (3, 0, 'idle'), (3, 4, 'idle'), (4, 1), (4, 3)], [(0, 1, 'idle'), (0, 3, 'idle'), (1, 0, 'idle'), (1, 4, 'idle'), (3, 0, 'idle'), (3, 4, 'idle'), (4, 1), (4, 3)], [(0, 1, 'idle'), (0, 3, 'idle'), (1, 0, 'idle'), (1, 4, 'idle'), (3, 0, 'idle'), (3, 4, 'idle'), (3, 1), (3, 3)], [(0, 1, 'idle'), (0, 3, 'idle'), (1, 0, 'idle'), (1, 4, 'idle'), (3, 0, 'idle'), (3, 4, 'idle'), (3, 1), (3, 2)], [(0, 1, 'idle'), (0, 3, 'idle'), (1, 0, 'idle'), (1, 4, 'idle'), (3, 0, 'idle'), (3, 4, 'idle'), (3, 1), (3, 1)], [(0, 1, 'idle'), (0, 3, 'idle'), (1, 0, 'idle'), (1, 4, 'idle'), (3, 0, 'idle'), (3, 4, 'idle'), (3, 1), (3, 1)], [(0, 1, 'idle'), (0, 3, 'idle'), (1, 0, 'idle'), (1, 4, 'idle'), (3, 0, 'idle'), (3, 4, 'idle'), (4, 1), (3, 2)], [(0, 1, 'idle'), (0, 3, 'idle'), (1, 0, 'idle'), (1, 4, 'idle'), (3, 0, 'idle'), (3, 4,

0.9656843878117398

In [2]:
with open('circuit_output.txt', 'w', encoding='utf-8') as f:
    for ops in scheduled_ops:
        # repr(ops) gives you something like:
        #   [('RX', -2.35619, 7), ...]
        f.write(repr(ops) + '\n')

In [5]:
import pennylane as qml
from pennylane import numpy as np

n = 8
dev = qml.device("default.qubit", wires=n)

def custom_qft(wires):
    """Forward QFT on `wires` (no final swap) using RY and CPhase."""
    for k in range(len(wires)):
        # Hadamard via RY(π/2)
        qml.RY(np.pi/2, wires=wires[k])
        # Controlled-phase rotations
        for j in range(k+1, len(wires)):
            angle = np.pi / (2 ** (j - k))
            # PennyLane’s CPhase is allowed—internally this is a native
            # controlled-RZ, but you can decompose RZ into RX/RY if you like:
            qml.CPhase(angle, wires=[wires[k], wires[j]])

def bit_reverse_state(state):
    """Reverse the bit-order of an n-qubit statevector."""
    N = state.size
    m = int(np.log2(N))
    out = np.zeros_like(state)
    for i in range(N):
        rev = int(format(i, f'0{m}b')[::-1], 2)
        out[rev] = state[i]
    return out

@qml.qnode(dev)
def U_custom():
    custom_qft(range(n))
    return qml.state()

@qml.qnode(dev)
def U_builtin():
    qml.QFT(wires=range(n))
    return qml.state()

# Execute
state_c = U_custom()
state_c_br = bit_reverse_state(state_c)
state_b = U_builtin()

fidelity = abs(np.vdot(state_c_br, state_b))**2
print(f"State fidelity vs. built-in QFT: {fidelity:.8f}")


State fidelity vs. built-in QFT: 1.00000000


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import imageio  # type: ignore
import os, warnings

# ------------------------------------------------------------------
# pos_hist       : list-of-lists or numpy array of shape (Tₚ, Q, 2) or (Tₚ, Q, 3)
#                  entries are (row, col) or (row, col, "idle")
# scheduled_ops  : list of length Tₛ, each a list of ops
# ------------------------------------------------------------------

scheduled_ops.append([])

# determine dimensions from list or array
Tp = len(pos_hist)
Q  = len(pos_hist[0])
Ts = len(scheduled_ops)
T  = min(Tp, Ts)

if Tp != Ts:
    warnings.warn(f"pos_hist has {Tp} ticks but scheduled_ops has {Ts}; "
                  f"animating first {T} ticks only.")

# compute grid extents
all_rows = []
all_cols = []
for frame in pos_hist[:T]:
    for entry in frame:
        r, c = entry[0], entry[1]
        all_rows.append(r)
        all_cols.append(c)
rows = max(all_rows) + 1
cols = max(all_cols) + 1

frames_dir = "frames"
os.makedirs(frames_dir, exist_ok=True)
frame_files = []

for t in range(T):
    # single-qubit ops in this tick
    sq_highlight = {
        q for name, *rest in scheduled_ops[t]
        if name in ("RX", "RY")
        for q in rest[-1:]
    }
    # MS qubits from previous tick
    ms_highlight = set()
    if t > 0:
        for name, *rest in scheduled_ops[t-1]:
            if name == "MS":
                ms_highlight.update(rest[-1])
    # idle qubits at this tick
    idle_highlight = {
        q for q in range(Q)
        if len(pos_hist[t][q]) == 3 and pos_hist[t][q][2] == "idle"
    }

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set_xlim(-0.5, cols - 0.5)
    ax.set_ylim(rows - 0.5, -0.5)
    ax.set_xticks(range(cols))
    ax.set_yticks(range(rows))
    ax.grid(True)
    ax.set_aspect("equal")

    for q in range(Q):
        entry = pos_hist[t][q]
        # unpack whether length 2 or 3
        r, c = entry[0], entry[1]

        if q in ms_highlight:
            ax.scatter(c, r, s=350, color="limegreen",
                       edgecolor="black", zorder=3)
            ax.text(c, r, str(q), color="white", weight="bold",
                    ha="center", va="center", zorder=4)

        elif q in sq_highlight:
            ax.scatter(c, r, s=350, color="red",
                       edgecolor="black", zorder=3)
            ax.text(c, r, str(q), color="white", weight="bold",
                    ha="center", va="center", zorder=4)

        elif q in idle_highlight:
            ax.scatter(c, r, s=300, color="skyblue",
                       edgecolor="black", zorder=3)
            ax.text(c, r, str(q), color="black", weight="bold",
                    ha="center", va="center", zorder=4)

        else:
            ax.scatter(c, r, s=200, facecolors="none",
                       edgecolors="black", zorder=2)
            ax.text(c, r, str(q), color="black",
                    ha="center", va="center")

    ax.set_title(f"Time step {t}")
    fname = os.path.join(frames_dir, f"frame_{t:03d}.png")
    fig.savefig(fname, dpi=100, bbox_inches="tight")
    plt.close(fig)
    frame_files.append(fname)

# Build GIF ------------------------------------------------------------------
gif_path = "movement.gif"
with imageio.get_writer(gif_path, mode="I", duration=2) as writer:
    for fname in frame_files:
        writer.append_data(imageio.imread(fname))

print(f"Saved animation to {gif_path}")


C:\Users\senli\AppData\Local\Temp\ipykernel_29252\1687954457.py:105: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  writer.append_data(imageio.imread(fname))


Saved animation to movement.gif
